# Natural Language Processing (NLP) with spaCy
In this notebook, we will use the Natural Language Processing library for Python called [spaCy][SPACY]. 
The functionality provided by [spaCy][SPACY] allows us to quickly extract parts-of-speech (POS) from text descriptions and to identify entities using [spaCy's][SPACY] named entity recognition (NER). We will initially  use [spaCy][SPACY]'s rule-based matching functionality to create a subject matcher that we can then apply to both Sinopia's RDF metadata as well as the associated full-text for a select sample of these resources.


[SPACY]: https://spacy.io/

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext lab_black

import datetime
import json
import string

import pathlib
import pandas as pd

import kglab
import rdflib
import helpers
import requests
import spacy

## Creating a FAST Panda DataFrame
Using a csv file derived from OCLC's [FAST](https://www.oclc.org/research/areas/data-science/fast.html)(Faceted Application of Subject Terminology) topic list, read into a Panda's DataFrame.

In [2]:
fast_topics = pd.read_csv("data/topic_uri_label_utf8.csv", names=["URL", "name"])

To get a sense of what data is in the `fast_topics` DataFrame we can look at the *shape*, *info*, and look at a random sample

In [3]:
print(f"Shape of fast_topics {fast_topics.shape}")
fast_topics.info()

Shape of fast_topics (460110, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460110 entries, 0 to 460109
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   URL     460110 non-null  object
 1   name    460110 non-null  object
dtypes: object(2)
memory usage: 7.0+ MB


In [5]:
fast_topics.sample(10)

,URL,name
177451,http://id.worldcat.org/fast/966085,Hypothermia--Age factors
258378,http://id.worldcat.org/fast/2004696,Bereavement in children--Religious aspects--Ju...
216383,http://id.worldcat.org/fast/1144390,Teachers--Supply and demand
113091,http://id.worldcat.org/fast/838764,Bridges--Live loads--Research
426694,http://id.worldcat.org/fast/1199140,Older blind people--Services for
64205,http://id.worldcat.org/fast/955915,Hichiriki
305361,http://id.worldcat.org/fast/1001184,Lobster industry
175604,http://id.worldcat.org/fast/948921,Guelfs and Ghibellines
157891,http://id.worldcat.org/fast/863423,Class actions (Civil procedure)--Trial practice
227690,http://id.worldcat.org/fast/1153283,Towers--Environmental aspects


From our sample, we see that the `name` has punctuation like **(),--** that will need to be removed later in our workflow.

## spaCy Phrase Matcher
With the [FAST][FAST] dataframe as our source, we now will create a [spaCy][SPACY] [PhraseMatcher](https://spacy.io/usage/rule-based-matching#phrasematcher) made up of phrases extracted from the dataframe using the [FAST][FAST] urls as identifiers.

We start by importing [spaCy][SPACY] base English vocabulary and create an empty natural language processing (nlp) pipeline that we pass into a new matcher object that will lower-case all of the [FAST][FAST] patterns.

[FAST]: https://www.oclc.org/research/areas/data-science/fast.html
[SPACY]: https://spacy.io/

In [8]:
from spacy.lang.en import English
from spacy.matcher import PhraseMatcher

nlp = English()
fast_topic_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")

In [6]:
"Rural population--Economic aspects--U.S. states".strip(string.punctuation).replace(
    "--", " "
).replace("(", "").replace(")", "").split()

['Rural', 'population', 'Economic', 'aspects', 'U.S.', 'states']

### Creatings Patterns
We will iterate through the dataframe and for each [FAST](https://www.oclc.org/research/areas/data-science/fast.html) series, using the **URL** as the identififer and for the name, remove punctuation, split each name into a list of words, and tokenize each term, and then add to the `fast_topic_matcher`. 

In [9]:
start = datetime.datetime.utcnow()
print(f"Started at {start} of FAST Matching")
# Iterate through the FAST Topic DataFrame
for row in fast_topics.iterrows():
    ident = row[1]["URL"]
    name_list = (
        row[1]["name"]
        .strip(string.punctuation)
        .replace("--", " ")
        .replace("(", "")
        .replace(")", "")
        .split()
    )
    name_list = [nlp.make_doc(row) for row in name_list]
    fast_topic_matcher.add(ident, name_list)
end = datetime.datetime.utcnow()
print(
    f"Finished adding FAST Topic patterns at {end}, total time {(end-start).seconds / 60.}"
)

Started at 2022-07-11 14:05:58.183584 of FAST Matching
Finished adding FAST Topic patterns at 2022-07-11 14:07:08.878073, total time 1.1666666666666667


Applying the spaCy Phrase matcher to titles and summaries from the Sinopia 

## Loading Sinopia Stage Knowledge Graph
Just like in the previous Jupyter notebook, we will load the saved knowledge graph that we created at the beginning and then we will query the graph using SPARQL.

In [10]:
stage_kg = kglab.KnowledgeGraph()
stage_kg.load_jsonld("data/stage.json")

In [11]:
len(stage_kg.rdf_graph())

138113

### RDF Literals Pandas DataFrame
With the Sinopia Stage knowledge graph loaded, we will query our Graph for *titles*, *labels*, and any *summary* triples and apply our `fast_topic_matcher` to the results.

> **NOTE** The `stage_text_nodes` dataframe has been saved in the **data** directory 
> and can be loaded for use instead of running this SPARQL query.

In [12]:
stage_text_nodes = stage_kg.query_as_df(
    """PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>

SELECT ?title ?label ?summary ?url

WHERE {
  OPTIONAL {
      ?title_bnode  bf:mainTitle ?title .
      ?url bf:title ?title_bnode .
  }
  OPTIONAL {
      ?label_bnode rdfs:label ?label .
      ?url bf:title ?label_bnode .
  }
  OPTIONAl {
      ?url bf:summary ?summary_bnode .
      ?summary_bnode rdfs:label ?summary  .
  }
}
"""
)

In [13]:
stage_text_nodes.to_json("data/stage-text-nodes.json")

In [14]:
stage_text_nodes.shape

(2108, 4)

In [15]:
stage_text_nodes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2108 entries, 0 to 2107
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    2108 non-null   object
 1   summary  155 non-null    object
 2   url      2108 non-null   object
 3   label    207 non-null    object
dtypes: object(4)
memory usage: 66.0+ KB


In [22]:
stage_text_nodes.sample(10)

,title,summary,url,label
1622,Work for trumpet and wind ensemble,NaN,<https://api.stage.sinopia.io/resource/88a78af...,NaN
1288,"Conspiracie and tragedie of Charles, Duke of B...",NaN,<https://api.stage.sinopia.io/resource/54da99a...,NaN
1169,The life and death of Mr. Badman,NaN,<https://api.stage.sinopia.io/resource/f48fd8e...,NaN
1450,Nugarum libri octo,NaN,<https://api.stage.sinopia.io/resource/74ac159...,NaN
951,Funerals of the high and mighty prince Henry,NaN,<https://api.stage.sinopia.io/resource/4ea28ad...,NaN
1357,Prose du Transibérien re-creation,A reproduction and adapatation of: Blaise Cend...,<https://api.stage.sinopia.io/resource/320aa1f...,NaN
722,Igrushki i zverushki,NaN,<https://api.stage.sinopia.io/resource/ca3a261...,NaN
202,"Hans Brinker, or, The silver skates",NaN,<https://api.stage.sinopia.io/resource/0970857...,NaN
1220,Vårt daglige ubehag,NaN,<https://api.stage.sinopia.io/resource/fbcb912...,NaN
1252,"Parallel1, title",NaN,<https://api.stage.sinopia.io/resource/0544eb0...,NaN


In [18]:
stage_text_nodes.tail()

,title,summary,url,label
2103,Religio medici,NaN,<https://api.stage.sinopia.io/resource/3ccea8b...,NaN
2104,Seven dog brothers,NaN,<https://api.stage.sinopia.io/resource/2841299...,NaN
2105,Alten Stettin (Map : 1588). German. 1617,NaN,<https://api.stage.sinopia.io/resource/8cf8b9d...,NaN
2106,De Arvlösa,NaN,<https://api.stage.sinopia.io/resource/111c0ea...,NaN
2107,Dirāsāt fī tārīkh Lībiyā al-muʻāṣir,NaN,<https://api.stage.sinopia.io/resource/a5963cf...,NaN


In [23]:
stage_text_nodes.iloc[1357].summary

"A reproduction and adapatation of: Blaise Cendrars' and Mme Delaunay-Terk's La prose du Transsibérien et de la Petite Jehanne de France published in Paris by Éditions des Hommes Nouveaux in 1913."

In [24]:
doc = nlp(stage_text_nodes.iloc[1357].summary)

In [25]:
len(fast_topic_matcher(doc))

171382

In [26]:
first_match = fast_topic_matcher(doc)[0]

In [27]:
first_match

(7524292931847503615, 0, 1)

In [28]:
nlp.vocab.strings[first_match[0]]

'http://id.worldcat.org/fast/793756'

In [29]:
fast_topics[fast_topics["URL"] == "http://id.worldcat.org/fast/793756"].iloc[0]["name"]

'A stars--Motion in line of sight'

In [30]:
stage_text_nodes.iloc[1578].title

'Pi︠a︡tiletka bez plana : Ukraina: 1991-1996 : formirovanie nat︠s︡ionalʹnogo gosudarstva, ekonomika, elity'

In [31]:
matches_1578 = helpers.get_matches(
    stage_text_nodes.iloc[1578].title, nlp, fast_topic_matcher, fast_topics, 25
)

In [32]:
len(matches_1578)

25

In [33]:
matches_1578

[('http://id.worldcat.org/fast/1053936', 'Pars plana'),
 ('http://id.worldcat.org/fast/1765673', 'Pars plana--Diseases'),
 ('http://id.worldcat.org/fast/1053937', 'Pars plana--Inflammation'),
 ('http://id.worldcat.org/fast/1152077',
  'Toffee (Fictitious character : Chambers)'),
 ('http://id.worldcat.org/fast/962697',
  'Huey (Fictitious character : Cameron)'),
 ('http://id.worldcat.org/fast/1740627',
  'Victorine (Fictitious character : Langlois)'),
 ('http://id.worldcat.org/fast/1744826',
  'Alison (Fictitious character : Chaucer)'),
 ('http://id.worldcat.org/fast/1100565',
  'Rosie (Fictitious character : Giff)'),
 ('http://id.worldcat.org/fast/951726',
  'Harriet (Fictitious character : Waddell)'),
 ('http://id.worldcat.org/fast/1899534',
  'Mansurah, Battle of, (Manṣūrah, Egypt : 1250) in literature'),
 ('http://id.worldcat.org/fast/1763560',
  'Edmund (Fictitious character : Shakespeare)'),
 ('http://id.worldcat.org/fast/1096909',
  'Rhapsody (Fictitious character : Haydon)'),
 (

The [spaCy][SPACY] `fast_topic_matcher` is greedy, meaning that it matches on any terms contained in the document. Changing the pattern matching strategy, and instead of parsing out each term in the FAST label, just create a single phrase with the `--` removed, adjusts what is matched.

[SPACY]: https://spacy.io/

In [34]:
strict_fast_topic_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")

In [35]:
start = datetime.datetime.utcnow()
print(f"Started at {start} for Strict FAST Matching")
# Iterate through the FAST Topic DataFrame
for row in fast_topics.iterrows():
    ident = row[1]["URL"]
    name_str = row[1]["name"].strip(string.punctuation).replace("--", " ")
    name_str = [
        nlp.make_doc(name_str),
    ]
    strict_fast_topic_matcher.add(ident, name_list)
end = datetime.datetime.utcnow()
print(
    f"Finished adding strict FAST Topic patterns at {end}, total time {(end-start).seconds / 60.}"
)

Started at 2022-07-11 14:21:06.619911 for Strict FAST Matching
Finished adding strict FAST Topic patterns at 2022-07-11 14:21:46.213066, total time 0.65


In [36]:
strict_matches_1578 = helpers.get_matches(
    stage_text_nodes.iloc[1578].title, nlp, strict_fast_topic_matcher, fast_topics, 25
)

In [37]:
strict_matches_1578

[]

## So the matcher may be too strict!

## spaCy Named Entity Recognition (NER)
One of the strengths of [spaCy][SPACY] is it's pre-existing models for named entity recognition (NER). In NER tasks, existing entities like people, locations, time periods are identified and tagged for later analysis. 

[SPACY]: https://spacy.io/

### Download model and Create NLP pipeline

To demonstrate, we will load a small English language model and perform NER on some of the *titles*, *labels*, and any *summary* triples contained in the `stage_text_nodes` dataframe.

In [38]:
! python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.5 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/Users/jpnelson/70-79 Presentations/70.02 RDF-AI Workshop - 02022 LD4 Conference/ld4-env/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [39]:
en_nlp = spacy.load("en_core_web_sm")

### Generate Documents and Display Entities
Using a different sample from the `stage_text_nodes` dataframe, we apply the `en_nlp` pipeline to some of these values to see if we can extract meaningful entities.

In [40]:
sample2 = stage_text_nodes.sample(10)

In [41]:
sample2

,title,summary,url,label
460,Indian no more,"In 1957, ten-year-old Regina Petit's Umpqua tr...",<https://api.stage.sinopia.io/resource/ee727dd...,NaN
635,Esprimere,NaN,<https://api.stage.sinopia.io/resource/19e55b5...,NaN
1748,"Market power, foreign direct investment and ju...",NaN,<https://api.stage.sinopia.io/resource/c874b3a...,"Market power, foreign direct investment and ju..."
5,"Uprooting, trauma, and confinement",NaN,<https://api.stage.sinopia.io/resource/be8059f...,NaN
1963,An enemy of the people,NaN,<https://api.stage.sinopia.io/resource/751cdc5...,NaN
1039,Fantasien,NaN,<https://api.stage.sinopia.io/resource/2c1beb6...,NaN
1811,test,NaN,<https://api.stage.sinopia.io/resource/e8eea4c...,NaN
1329,Politics of birth.,NaN,<https://api.stage.sinopia.io/resource/001493d...,NaN
174,Psychology and life,NaN,<https://api.stage.sinopia.io/resource/ac02260...,NaN
712,May-day,NaN,<https://api.stage.sinopia.io/resource/cd14d2b...,NaN


In [42]:
def show_entities(index_number, column):
    print(f"Entities for {index_number}")
    doc = en_nlp(stage_text_nodes.iloc[index_number][column])
    for ent in doc.ents:
        print(f"\t{ent}, {ent.label_}")
    return doc

In [48]:
stage_text_nodes.iloc[1748]["title"]

'Market power, foreign direct investment and judicial reform in Russia and Kazakhstan'

In [54]:
doc_1748_title = show_entities(1748, "title")

Entities for 1748
	Russia, GPE
	Kazakhstan, GPE


In [50]:
doc_460_summary = show_entities(460, "summary")

Entities for 460
	1957, DATE
	ten-year-old, DATE
	Regina Petit's, PERSON
	Umpqua, ORG
	Oregon, GPE
	Los Angeles, GPE
	Indian, NORP


### spaCy NER and POS Visualizations
The [spaCy][SPACY] project provides the `displacy` class that has two helpful visualizers. The first **ner** displays the identified entities in context of the text and the second identifies the parts-of-speech and how they are related to each other in the **dep** option. 

[SPACY]: https://spacy.io/

In [51]:
from spacy import displacy

displacy.render(doc_460_summary, jupyter=True, style="ent")

In [52]:
spacy.explain("NORP")

'Nationalities or religious or political groups'

In [55]:
displacy.render(doc_1748_title, jupyter=True, style="ent")

## Exercise 1
Download and create a `nlp` pipeline using the *en_core_web_sm* model and see what entities are identified using your sample from the `stage_text_nodes` dataframe.

> **Extra credit** Find a non-english resource in `stage_text_nodes`, find a corresponding 
> [spaCy][SPACY] trained pipeline at https://spacy.io/models and follow the above steps to 
> identify any entities in the *title*, *label*, or *summary* values.

[SPACY]: https://spacy.io/

## Full-Text and Sinopia RDF Metadata
Exciting possibilities of applying machine learning to library workflows comes when we combine linked-data cataloging with the availability of the full-text. For example, we can run NER on the entirety of the full-text, and add any identified entities to our RDF. 

> Special thanks to Tim Thompson and Greta de Groat for providing examples of cataloged
> resources in Sinopia that have available full-text!


### Example One: *Alarmingly suspicious*
First we will download the Sinopia record for this Work, https://api.sinopia.io/resource/65a2b059-5ac1-48a6-adbb-870712c3060c, and extract the `data` property to create a RDF graph.

In [56]:
example1_result = requests.get(
    "https://api.sinopia.io/resource/65a2b059-5ac1-48a6-adbb-870712c3060c"
)
example1_graph = rdflib.Graph()
for ns, url in helpers.NAMESPACES.items():
    example1_graph.namespace_manager.bind(ns, url)
example1_graph.parse(
    data=json.dumps(example1_result.json().get("data")), format="json-ld"
)

<Graph identifier=Nce01e088acd74bec9adced3a7a0c85bd (<class 'rdflib.graph.Graph'>)>

In [57]:
print(example1_graph.serialize(format="turtle"))

@prefix bf: <http://id.loc.gov/ontologies/bibframe/> .
@prefix bflc: <http://id.loc.gov/ontologies/bflc/> .
@prefix ns1: <http://www.europeana.eu/schemas/edm/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sinopia: <http://sinopia.io/vocabulary/> .

<https://api.sinopia.io/resource/65a2b059-5ac1-48a6-adbb-870712c3060c> a bf:Work,
        <http://share-vde.org/rdfBibframe/SuperWork> ;
    rdfs:label "Simpson, J. Palgrave (John Palgrave), 1807-1887. Alarmingly suspicious"@eng ;
    bf:adminMetadata [ a bf:AdminMetadata ;
            bflc:catalogerId "tt434"@eng ;
            bflc:encodingLevel <http://id.loc.gov/vocabulary/menclvl/f> ;
            bf:descriptionConventions <https://id.loc.gov/vocabulary/descriptionConventions/rda> ;
            bf:descriptionLanguage <https://id.loc.gov/vocabulary/languages/eng> ;
            bf:descriptionModifier <https://id.loc.gov/vocabulary/organizations/cty> ;
            bf:source

Second, we open the full-text that was retrieved from the [Haithtrust](https://babel.hathitrust.org/cgi/pt?id=uiuo.ark:/13960/t14n5kz7q&view=1up&seq=3) and load all of the text and save to the `example1_text` variable. 

In [58]:
with open("data/65a2b059-5ac1-48a6-adbb-870712c3060c.txt") as fo:
    example1_text = fo.read()

print(f"Number of characters in the text is {len(example1_text):,}")

Number of characters in the text is 61,249


In [60]:
example1_text[30_000:30_250]

'nouncing) Luncheon is on the table ! \n \n Em. At last, Colonel 1 \n \n The Colonel, goes up stage grumbling, pays no heed. JSir Barnabas advances to \n offer Lady Emily his arm; Sharp springs in before him vnth assiduous gal- \n lantry. Lady Emily accepts'

Third, we will run the full-text through our [spaCy][SPACY] en_nlp pipeline and then examine the identified entities.

[SPACY]: https://spacy.io/

In [61]:
example1_doc = en_nlp(example1_text)
print(f"Number of entities: {len(example1_doc.ents):,}")

Number of entities: 587


Of the 587 entities, we will extract the **PERSON** and **GPE** (location) entities for further analysis. 

In [62]:
people, locations = [], []

for ent in example1_doc.ents:
    if ent.label_.startswith("PERSON"):
        people.append(ent)
    if ent.label_.startswith("GPE"):
        locations.append(ent)
print(f"NER found {len(people):,} people and {len(locations):,} locations")

NER found 281 people and 34 locations


Looking at the `people` list, we notice a number of duplications (for now we don't care where the person entity was found in the text) so we filter this list.

In [63]:
people

[J.
 	   Palgrave Simpson,
 J. Palgrave,
 John Palgrave,
 Si y 
  
  1 -,
 Att'a 
  ,
 Cumin Man 
  ,
 tifW 
  
  Tim,
 tu-Ujjjg,
 Smith,
 Paddy Miles,
 Depvfcnt Watet,
 k Dur-Room,
 \ 
  
  Won,
 English Theatres,
 Augustus Sharp,
 Mary Mum,
 Belton 
  
  .Mr,
 Basil Baker 
  
  ,
 Henry Butler,
 De,
 Laura Keene,
 Frank Matthews,
 L.,
 Barnabas,
 Lady Buzzbite,
 Mary Mum,
 Lady B.,
 Mary Mum,
 Lady B. Lady Emily,
 Lady B.,
 Lady Emily,
 Lady B. Alone,
 Lady B.,
 Lady B. Ah,
 Lady B.,
 Barnabas,
 Lady Buzzbite,
 Lady B.,
 Lady B. Well,
 Barnabas,
 Barnabas,
 Barna¬ 
  ,
 Mum,
 Lady B.,
 Lady B.,
 Lady B. B,
 Lady B. After,
 Lady B.,
 Lady B. Colonel,
 Lady B. He,
 Lady B.,
 Barnabas,
 Lady B.,
 Lady Emily,
 Lady B.,
 Lady Emily's,
 Barnabas,
 Barnabas,
 Barnabas,
 Barnabas,
 Barna¬ 
  bas,
 Barnabas,
 Barnabas,
 Barnabas,
 Lady B.,
 Lady Buzzbite,
 Barnabas,
 Lady B. Ah,
 Lady Emily,
 Barnabas,
 Barnabas,
 Barnabas,
 Lady Buzzbite,
 Bar¬ 
  ,
 Lady Emily,
 Lady B.,
 Lady Buzzbite,
 La

In [64]:
people = set([" ".join(ent.text.split()) for ent in people])
locations = set([" ".join(ent.text.split()) for ent in locations])

In [65]:
len(people), len(locations)

(103, 20)

With these identified names, we can use the VIAF [SRU Search](https://platform.worldcat.org/api-explorer/apis/VIAF/AuthorityCluster/SRUSearch) on each of the `people` entities to see if we can narrow down our list using this authority source. In the helpers module, the `viaf_people_search` function returns a dictionary VIAF urls that match the term and the first main heading of the record.

We can then aggregate these results and see what remains. 

In [66]:
example1_people = {}
for i, term in enumerate(list(people)):
    # Removes newlines, tabs, and spaces from the entity text
    viaf_result = helpers.viaf_people_search(term)
    example1_people.update(viaf_result)
    if not i % 10 and i > 0:
        print(".", end="")
    if not i % 25:
        print(f"{i}", end="")

0..25...50..75...100

In [44]:
example1_people

{'http://viaf.org/viaf/4819162906466578110009/': ['Ball, John'],
 'http://viaf.org/viaf/9924151247972544270006/': ['Voigt, Emil 1881-'],
 'http://viaf.org/viaf/9874158792833139040004/': ['Johan Georg Frederik Ræder juriste danois'],
 'http://viaf.org/viaf/138389/': ['Sharp, Tony.'],
 'http://viaf.org/viaf/310152742908027731493/': ['Valletort, Lady 19.sc'],
 'http://viaf.org/viaf/272152742908127731486/': ['Valletort, Lord'],
 'http://viaf.org/viaf/26152742893627730540/': ['Popeley, William 1718+'],
 'http://viaf.org/viaf/9994156012397449700001/': ['Schneider, Raimund'],
 'http://viaf.org/viaf/9985152684042123430003/': ['Schneider, Konrad'],
 'http://viaf.org/viaf/9983160668220203560000/': ['Schneider, Wolfgang 1941-'],
 'http://viaf.org/viaf/9995160909265912870007/': ['Look, Emily'],
 'http://viaf.org/viaf/9978165628873142480009/': ['Hyatt, Emily'],
 'http://viaf.org/viaf/9972162669668555500005/': ['La Trobe-Bateman, Emily'],
 'http://viaf.org/viaf/9999160668126703560006/': ['Engel, Wol

### Example 2 - *Seismic performance assessment of buildings*
The second example is the report, *Seismic performance assessment of buildingss*, described by the following Sinopia Resources:

- BIBFRAME Work https://api.stage.sinopia.io/resource/2ffc86d1-4850-4e4b-974c-49ded8ce4b3f
- BIBFRAME Instance https://api.stage.sinopia.io/resource/e865acc1-9b24-401e-a0eb-36ca1ca8b1d6


First we will retrieve these resources and add them to a RDF graph.

In [45]:
example2_graph = rdflib.Graph()
for ns, url in helpers.NAMESPACES.items():
    example2_graph.namespace_manager.bind(ns, url)

for sinopia_url in [
    "https://api.stage.sinopia.io/resource/2ffc86d1-4850-4e4b-974c-49ded8ce4b3f",
    "https://api.stage.sinopia.io/resource/e865acc1-9b24-401e-a0eb-36ca1ca8b1d6",
]:
    sinopia_result = requests.get(sinopia_url)
    example2_graph.parse(
        data=json.dumps(sinopia_result.json()["data"]), format="json-ld"
    )
print(f"Total number of triples for example 2 {len(example2_graph):,}")

Total number of triples for example 2 152


Second, we will open and read the full-text for *Improving water supply networks*

In [46]:
with open("data/e865acc1-9b24-401e-a0eb-36ca1ca8b1d6.txt") as fo:
    example2_text = fo.read()

print(f"Number of characters in the text is {len(example2_text):,}")

Number of characters in the text is 671,016


Third, we will run the full-text through our [spaCy][SPACY] en_nlp pipeline and then examine the identified entities.

[SPACY]: https://spacy.io

In [47]:
example2_doc = en_nlp(example2_text)
print(f"Number of entities: {len(example2_doc.ents):,}")

Number of entities: 10,220


In [48]:
ent_types = {}
for ent in example2_doc.ents:
    if ent.label_ in ent_types:
        ent_types[ent.label_] += 1
    else:
        ent_types[ent.label_] = 1

In [49]:
ent_types

{'ORG': 1820,
 'WORK_OF_ART': 324,
 'PRODUCT': 672,
 'CARDINAL': 4170,
 'GPE': 686,
 'PERSON': 685,
 'FAC': 42,
 'DATE': 533,
 'MONEY': 136,
 'ORDINAL': 138,
 'NORP': 34,
 'QUANTITY': 36,
 'PERCENT': 794,
 'LAW': 61,
 'LOC': 22,
 'TIME': 62,
 'EVENT': 5}

In [50]:
example2_people, example2_locations = [], []

for ent in example2_doc.ents:
    if ent.label_.startswith("PERSON"):
        example2_people.append(ent)
    if ent.label_.startswith("GPE"):
        example2_locations.append(ent)
print(
    f"NER found {len(example2_people):,} people and {len(example2_locations):,} locations"
)

NER found 685 people and 686 locations


In [51]:
example2_people[90].start, example2_people[90].end

(639, 641)

In [52]:
example2_people[90]

Christopher Higgins

We now de-duplicate `example2_people` and `example2_locations`example2_people

In [53]:
example2_people = set([" ".join(ent.text.split()) for ent in example2_people])
example2_locations = set([" ".join(ent.text.split()) for ent in example2_locations])

In [54]:
len(example2_people)

359

In [55]:
type(list(example2_people)[52])

str

In [56]:
for person in list(example2_people)[50:100]:
    print(person)

Vamvatsikos
P. Benson Shing
Christopher Higgins
max min characterization
B10
ASTM
Appendix K
E2022.112 Filing
.................................................................... G-2
Kramer
Richard Behr
Appendix J.
Deborah Beck
Duy Vu
Stojadinovic
Peter Morris
r.
Hussein
Earthquake Spectra
Finishes
Juan Murcia-Delso
Robert P. Kennedy
Xibei Xincun
D. Jared DeBock
Seismic Hazard and Risk Analysis
Baker
Ronald Hamburger
Ross
m.

D402 Wet
Christopher Smith
Keith
Daniel Saldana
Drirfitf
Chi-Chi
Bruce R. Ellingwood
C. Allin Cornell
Nirmal Kumawat Udit S. Tambe
Angie Harris
Toro
William O’Brien
  h  ln(H
Bob Bachman
J.4
John Hooper
Jack Moehle
Part II
Scott Hagie Angie Harris
Masonry Walls


In [57]:
len(example2_locations)

198

In [58]:
for location in list(example2_locations)[0:50]:
    print(location)

Denver
U.S
SF
Deierlein
A.S.
Medina
New York 14223
Bertero
S.L.
Christopoulos
Moehle
San Mateo
FRAGILITY
B.S.J.
La Jolla
South Carolina
Tehran
Vancouver
Reston
N-S
Escondido
W.J.
New Jersey
Illinois
Stanford
Nevada
Vairo
T.Y.
Michigan
Somerville
Whittaker
Corvallis
America
Longmont
−
Hills
Ohio
Hebron
B.7
Tobin
Missouri
∆e
Coef
Cascadia
Carson
Krawinkler
Loads
Youngs
Housner
Benjamin


## Exercise 2
For the following Sinopia Resources, create a graph, and a NLP pipeline to identify any people and locations in the full-text. The full-text is available in `data/a90f911e-d03e-427f-a36e-ed1e3d1a9b3b.txt` file.

- BIBFRAME Work https://api.stage.sinopia.io/resource/d1e377d0-9ca6-4377-a9a1-a20303c5fe66
- BIBFRAME Instance https://api.stage.sinopia.io/resource/a90f911e-d03e-427f-a36e-ed1e3d1a9b3b

# Extra Credit

## Custom NER Ruler
Another approach we can take is to build a custom [Named Entity Ruler](https://spacy.io/usage/rule-based-matching#entityruler) for the FAST Topics.  

In [59]:
nlp = spacy.load("en_core_web_sm")
nlp.disable_pipe("ner")
fast_ruler = nlp.add_pipe("entity_ruler")

In [60]:
start = datetime.datetime.utcnow()
print(f"Started at {start} for FAST Entities Matching")
all_patterns = []
# Iterate through the FAST Topic DataFrame
for row in fast_topics.iterrows():
    ident = row[1]["URL"]
    name_list = row[1]["name"].strip(string.punctuation).split("--")
    fast_patterns = []
    for name in name_list:
        pattern = []
        terms = name.split(" ")
        for term in terms:
            term = term.strip()
            if len(term) < 1:
                continue
            pattern.append({"LOWER": term.lower()})
        fast_patterns.append(pattern)
    all_patterns.append({"label": row[1]["name"],
                         "pattern": fast_patterns})
    current_count = len(all_patterns)
    if not current_count%100 and current_count > 0:
        print(".", end="")
    if not current_count%1000:
        print(f"{current_count:,}", end="")
fast_ruler.add_patterns(all_patterns)
end = datetime.datetime.utcnow()
print(
    f"Finished adding FAST Patterns at {end}, total time {(end-start).seconds / 60.}"
)

Started at 2022-07-10 23:16:47.009660 for FAST Entities Matching
..........1,000..........2,000..........3,000..........4,000..........5,000..........6,000..........7,000..........8,000..........9,000..........10,000..........11,000..........12,000..........13,000..........14,000..........15,000..........16,000..........17,000..........18,000..........19,000..........20,000..........21,000..........22,000..........23,000..........24,000..........25,000..........26,000..........27,000..........28,000..........29,000..........30,000..........31,000..........32,000..........33,000..........34,000..........35,000..........36,000..........37,000..........38,000..........39,000..........40,000..........41,000..........42,000..........43,000..........44,000..........45,000..........46,000..........47,000..........48,000..........49,000..........50,000..........51,000..........52,000..........53,000..........54,000..........55,000..........56,000..........57,000..........58,000..........59,000

ValueError: [E154] One of the attributes or values is not supported for token patterns. Please use the option `validate=True` with the Matcher, PhraseMatcher, or EntityRuler for more details.